In [58]:
!ls -l /kaggle/input/translate6

total 60
-rw-r--r-- 1 nobody nogroup  3513 Aug 25 00:48 analisis.py
-rw-r--r-- 1 nobody nogroup  1109 Aug 25 00:48 attention.py
-rw-r--r-- 1 nobody nogroup  1521 Aug 25 00:48 decoder.py
-rw-r--r-- 1 nobody nogroup   908 Aug 25 00:48 encoder.py
-rw-r--r-- 1 nobody nogroup  9298 Aug 25 00:48 eval.py
-rw-r--r-- 1 nobody nogroup  1738 Aug 25 00:48 heatmap.py
-rw-r--r-- 1 nobody nogroup 11924 Aug 25 00:48 main.py
-rw-r--r-- 1 nobody nogroup  3754 Aug 25 00:48 seq2seq.py
-rw-r--r-- 1 nobody nogroup  2035 Aug 25 00:48 top_words.py
-rw-r--r-- 1 nobody nogroup  5281 Aug 25 00:48 util.py


In [59]:
import unicodedata
from collections import Counter
from pathlib import Path
import argparse
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import math
import sacrebleu
import sys
import os

# Definisikan konstanta global di luar fungsi
SPECIALS = ["<pad>", "<bos>", "<eos>", "<unk>"]
PAD, BOS, EOS, UNK = range(4)

# --- Class Attention, Encoder, Decoder (sebagai contoh, pastikan Anda memiliki implementasi yang benar) ---
class BahdanauAttentionQKV(nn.Module):
    def __init__(self, hidden_size, query_size, key_size, dropout_p=0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.Wa = nn.Linear(query_size, hidden_size)
        self.Wk = nn.Linear(key_size, hidden_size)
        self.V = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self, query, keys, mask=None):
        # query: (batch_size, 1, hidden_size)
        # keys: (batch_size, seq_len, hidden_size)
        query = self.Wa(query)
        keys = self.Wk(keys)
        scores = self.V(torch.tanh(query + keys))
        scores = scores.squeeze(-1)
        if mask is not None:
            scores.masked_fill_(mask, -float("inf"))
        return scores
        
class BahdanauEncoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout_p=0.1):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=PAD)
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, bidirectional=True)
        self.fc = nn.Linear(encoder_hidden_dim * 2, decoder_hidden_dim)
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self, src):
        src_seq_len = src.shape[0]
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        return outputs, hidden

class BahdanauDecoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, attention, dropout_p=0.1):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, embedding_dim, padding_idx=PAD)
        self.rnn = nn.GRU(embedding_dim + encoder_hidden_dim * 2, decoder_hidden_dim)
        self.fc_out = nn.Linear(embedding_dim + decoder_hidden_dim + encoder_hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(p=dropout_p)
        
    def forward(self, x, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(x))
        
        encoder_outputs_T = encoder_outputs.transpose(0, 1)
        
        query_for_attn = hidden.unsqueeze(1)
        
        attn_weights = self.attention(query_for_attn, encoder_outputs_T)
        attn_weights = F.softmax(attn_weights, dim=1)
        weighted_context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs_T).squeeze(1)

        rnn_input = torch.cat((embedded.squeeze(0), weighted_context), dim=1).unsqueeze(0)
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        output = output.squeeze(0)
        embedded = embedded.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted_context, embedded), dim=1))
        
        return prediction.unsqueeze(0), hidden.squeeze(0), attn_weights

# ---- Class Model Seq2Seq dengan Beam Search ----
class BahdanauSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, pad_id, bos_id, eos_id):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.pad_id = pad_id
        self.bos_id = bos_id
        self.eos_id = eos_id

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len, batch_size = trg.size()
        outputs = torch.zeros(trg_len, batch_size, self.decoder.output_dim).to(self.device)
        encoder_outputs, hidden = self.encoder(src)
        trg_input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden, _ = self.decoder(trg_input.unsqueeze(0), hidden, encoder_outputs)
            outputs[t] = output.squeeze(0)
            
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(2).squeeze(0)
            trg_input = trg[t] if teacher_force else top1
            
        return outputs, None

    def greedy_decode(self, src, max_len=40):
        batch_size = src.size(1)
        encoder_outputs, hidden = self.encoder(src)
        ys = torch.ones(1, batch_size, dtype=torch.long).fill_(self.bos_id).to(self.device)
        
        for _ in range(max_len - 1):
            y_tm1 = ys[-1].unsqueeze(0)
            output, hidden, _ = self.decoder(y_tm1, hidden, encoder_outputs)
            
            next_word_id = output.argmax(2)
            
            ys = torch.cat([ys, next_word_id], dim=0)

        return ys, None

    def beam_search_decode(self, src, max_len=40, beam_size=3):
        batch_size = src.size(1)
        encoder_outputs, hidden = self.encoder(src)
        
        hypotheses = torch.ones(1, batch_size, beam_size, dtype=torch.long).fill_(self.bos_id).to(self.device)
        hyp_scores = torch.zeros(batch_size, beam_size).to(self.device)
        
        hidden_beams = hidden.unsqueeze(1).repeat(1, beam_size, 1)
        hidden_beams = hidden_beams.view(batch_size * beam_size, -1)
        
        encoder_outputs_beams = encoder_outputs.transpose(0, 1).unsqueeze(1).repeat(1, beam_size, 1, 1).view(batch_size * beam_size, encoder_outputs.size(0), -1).transpose(0,1)

        for _ in range(max_len - 1):
            last_tokens = hypotheses[-1].view(-1, 1).transpose(0,1)
            output, hidden_beams, _ = self.decoder(last_tokens, hidden_beams.view(batch_size * beam_size, -1), encoder_outputs_beams)
            output = output.transpose(0, 1)
            output = F.log_softmax(output, dim=-1)
            
            cand_scores = hyp_scores.unsqueeze(2) + output.view(batch_size, beam_size, -1)
            
            cand_scores, cand_indices = cand_scores.view(batch_size, -1).topk(beam_size, dim=-1)
            
            hyp_scores = cand_scores
            
            prev_hyp_indices = cand_indices // self.decoder.output_dim
            new_token_indices = cand_indices % self.decoder.output_dim
            
            new_hypotheses = torch.zeros(hypotheses.size(0) + 1, batch_size, beam_size, dtype=torch.long).to(self.device)
            for i in range(hypotheses.size(0)):
                new_hypotheses[i] = torch.gather(hypotheses[i], 1, prev_hyp_indices)
            new_hypotheses[-1] = new_token_indices
            
            hypotheses = new_hypotheses
            
            eos_mask = (new_token_indices == self.eos_id)
            if eos_mask.all():
                break

        best_hyp_indices = hyp_scores.argmax(dim=1)
        final_hypotheses = torch.zeros(max_len, batch_size, dtype=torch.long).to(self.device)

        for b in range(batch_size):
            best_hyp = hypotheses[:, b, best_hyp_indices[b]]
            final_hypotheses[:len(best_hyp), b] = best_hyp
            final_hypotheses[len(best_hyp):, b] = self.pad_id

        return final_hypotheses.to(self.device)
        
# ---- Helper functions for data processing and evaluation ----
def normalize(text):
    return unicodedata.normalize("NFKC", text.lower().strip())

def to_ids(tokens, vocab, unk_id=3, bos_id=1, eos_id=2):
    ids = [bos_id]
    for tok in tokens:
        ids.append(vocab.get(tok, unk_id))
    ids.append(eos_id)
    return ids

def decode_ids(ids, itos, bos_id=1, eos_id=2):
    tokens = []
    for i in ids:
        if i.item() == eos_id:
            break
        if i.item() != bos_id:
            tokens.append(itos[i.item()])
    return " ".join(tokens)

def collate_batch(batch):
    src_list, trg_list = [], []
    for src, trg in batch:
        src_list.append(src)
        trg_list.append(trg)
    src_padded = torch.nn.utils.rnn.pad_sequence(src_list, batch_first=True, padding_value=PAD)
    trg_padded = torch.nn.utils.rnn.pad_sequence(trg_list, batch_first=True, padding_value=PAD)
    return src_padded, trg_padded

def load_pairs(file_path, max_len=20, max_pairs=None):
    pairs = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if max_pairs and i >= max_pairs: break
            parts = line.strip().split('\t')
            src, trg = normalize(parts[0]).split(' '), normalize(parts[1]).split(' ')
            if len(src) < max_len and len(trg) < max_len:
                pairs.append((src, trg))
    return pairs

def split_pairs(pairs, train_ratio=0.8, val_ratio=0.1):
    n = len(pairs)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)
    return pairs[:n_train], pairs[n_train:n_train + n_val], pairs[n_train + n_val:]

def build_vocab(token_lists, min_freq=1, max_size=None, specials=SPECIALS):
    counter = Counter()
    for toks in token_lists:
        counter.update(toks)
    filtered = [(w, c) for w, c in counter.items() if c >= min_freq]
    filtered.sort(key=lambda x: (-x[1], x[0]))
    if max_size is not None:
        filtered = filtered[:max(0, max_size - len(specials))]
    vocab = {sp: i for i, sp in enumerate(specials)}
    for w, _ in filtered:
        if w not in vocab:
            vocab[w] = len(vocab)
    itos = {i: w for w, i in vocab.items()}
    return vocab, itos

def epoch_run(model, loader, criterion, optimizer, train=True, teacher_forcing=0.5):
    model.train() if train else model.eval()
    total_loss, total_tokens = 0.0, 0
    device = next(model.parameters()).device
    
    with torch.set_grad_enabled(train):
        for src, trg in tqdm(loader):
            src = src.to(device).T
            trg = trg.to(device).T
            
            outputs, _att = model(src, trg, teacher_forcing_ratio=teacher_forcing if train else 0.0)
            
            logits = outputs[1:].reshape(-1, outputs.size(-1))
            target = trg[1:].reshape(-1)
            
            loss = criterion(logits, target)
            
            if train:
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            n_tokens = (target != PAD).sum().item()
            total_loss += loss.item() * n_tokens
            total_tokens += n_tokens
            
    avg_loss = total_loss / max(1, total_tokens)
    ppl = math.exp(avg_loss) if avg_loss < 20 else float("inf")
    return avg_loss, ppl

def evaluate_sacrebleu(model, loader, trg_itos=None, sp_trg=None, beam_size=1):
    model.eval()
    refs, hyps = [], []
    with torch.no_grad():
        for src, trg in tqdm(loader):
            src, trg = src.to(model.device).T, trg.to(model.device).T
            if beam_size > 1:
                ys = model.beam_search_decode(src, max_len=40, beam_size=beam_size)
            else:
                ys, _ = model.greedy_decode(src, max_len=40)
            
            if sp_trg:
                pass
            else:
                for y in ys.T.tolist():
                    hyps.append(decode_ids(torch.tensor(y), trg_itos))
            
            if sp_trg:
                pass
            else:
                for t in trg.T.tolist():
                    refs.append(decode_ids(torch.tensor(t), trg_itos))
    
    refs_sacrebleu = [[ref] for ref in refs]
    bleu = sacrebleu.corpus_bleu(hyps, refs_sacrebleu).score
    return bleu

# ---- NMTDataset class ----
class NMTDataset(Dataset):
    def __init__(self, pairs, src_vocab, trg_vocab):
        self.data = [(to_ids(src, src_vocab), to_ids(trg, trg_vocab)) for src, trg in pairs]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        src_ids, trg_ids = self.data[idx]
        return torch.tensor(src_ids, dtype=torch.long), torch.tensor(trg_ids, dtype=torch.long)

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_path', type=str, default='/kaggle/input/translate5/ind-eng/ind.txt', help='Path to txt data')
    parser.add_argument('--epochs', type=int, default=10, help='Number of training epochs')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size')
    parser.add_argument('--lr', type=float, default=1e-3, help='Learning rate')
    parser.add_argument('--tf', type=float, default=0.5, help='Teacher Forcing')
    parser.add_argument('--dropout', type=float, default=0.15, help='dropout')
    parser.add_argument('--max_vocab', type=int, default=None)
    parser.add_argument('--target_lang', type=str, default='ID', help='Bahasa tujuan')
    parser.add_argument('--checkpoint', type=str, default='bahdanau_best.pt', help='Path to save model checkpoint')
    
    args, unknown = parser.parse_known_args()

    data_file = Path(args.data_path)
    pairs = load_pairs(data_file, max_len=20, max_pairs=None)
    print(f"Total usable pairs after filtering: {len(pairs):,}")
    train_pairs, val_pairs, test_pairs = split_pairs(pairs, 0.8, 0.1)
    print(f"Train: {len(train_pairs):,}, Val: {len(val_pairs):,}, Test: {len(test_pairs):,}")

    en_vocab, en_itos = build_vocab([src for src, _ in train_pairs], max_size=args.max_vocab)
    id_vocab, id_itos = build_vocab([tgt for _, tgt in train_pairs], max_size=args.max_vocab)

    with open("en_vocab.json", "w") as f:
        json.dump(en_vocab, f)
    with open("id_vocab.json", "w") as f:
        json.dump(id_vocab, f)

    print(f"EN vocab size: {len(en_vocab):,} | ID vocab size: {len(id_vocab):,}")
    
    train_ds = NMTDataset(train_pairs, en_vocab, id_vocab)
    val_ds = NMTDataset(val_pairs, en_vocab, id_vocab)
    test_ds = NMTDataset(test_pairs, en_vocab, id_vocab)

    BATCH_SIZE = args.batch_size
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
    
    ENCODER_HIDDEN_SIZE = 512
    DECODER_HIDDEN_SIZE = 256
    ENCODER_EMBEDDING_DIM = 256
    DECODER_EMBEDDING_DIM = 256

    encoder = BahdanauEncoder(input_dim=len(en_vocab), embedding_dim=ENCODER_EMBEDDING_DIM, encoder_hidden_dim=ENCODER_HIDDEN_SIZE, decoder_hidden_dim=DECODER_HIDDEN_SIZE, dropout_p=args.dropout)
    attn = BahdanauAttentionQKV(hidden_size=DECODER_HIDDEN_SIZE, query_size=DECODER_HIDDEN_SIZE, key_size=2 * ENCODER_HIDDEN_SIZE, dropout_p=0.0)
    decoder = BahdanauDecoder(output_dim=len(id_vocab), embedding_dim=DECODER_EMBEDDING_DIM, encoder_hidden_dim=ENCODER_HIDDEN_SIZE, decoder_hidden_dim=DECODER_HIDDEN_SIZE, attention=attn, dropout_p=args.dropout)
    
    seq2seq = BahdanauSeq2Seq(encoder, decoder, device, pad_id=PAD, bos_id=BOS, eos_id=EOS).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=PAD)
    optimizer = torch.optim.Adam(seq2seq.parameters(), lr=args.lr)
    
    history = {"train_loss": [], "val_loss": [], "train_ppl": [], "val_ppl": [], "val_bleu": []}
    EPOCHS = args.epochs
    best_val = float("inf")
    
    for epoch in range(1, EPOCHS + 1):
        tf = max(0.3, 0.7 - 0.04 * (epoch - 1))
        train_loss, train_ppl = epoch_run(seq2seq, train_loader, criterion, optimizer, train=True, teacher_forcing=tf)
        val_loss, val_ppl = epoch_run(seq2seq, val_loader, criterion, optimizer, train=False, teacher_forcing=0.0)
        
        val_bleu = evaluate_sacrebleu(seq2seq, val_loader, trg_itos=id_itos)
        
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_ppl"].append(train_ppl)
        history["val_ppl"].append(val_ppl)
        history["val_bleu"].append(val_bleu)

        print(f"Epoch {epoch:02d} | TF={tf:.2f} | Train Loss {train_loss:.4f} PPL {train_ppl:.2f} | Val Loss {val_loss:.4f} PPL {val_ppl:.2f} | Val Bleu {val_bleu:.4f} ")

        if val_loss < best_val:
            best_val = val_loss
            torch.save(seq2seq.state_dict(), args.checkpoint)
            print("Saving best to", args.checkpoint)

    with open("train_history.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["epoch", "train_loss", "val_loss", "train_ppl", "val_ppl", "val_bleu"])
        for i in range(EPOCHS):
            w.writerow([i + 1, history["train_loss"][i], history["val_loss"][i], history["train_ppl"][i], history["val_ppl"][i], history["val_bleu"][i]])

    seq2seq.load_state_dict(torch.load(args.checkpoint, map_location=device))
    test_loss, test_ppl = epoch_run(seq2seq, test_loader, criterion, optimizer, train=False, teacher_forcing=0.0)
    
    test_bleu = evaluate_sacrebleu(seq2seq, test_loader, trg_itos=id_itos, beam_size=3) 
    print(f"TEST | Loss {test_loss:.4f} | PPL {test_ppl:.2f} | SacreBLEU {test_bleu:.2f}")

    references = []
    hypotheses = []
    seq2seq.eval()
    with torch.no_grad():
        n_show = 5
        shown = 0
        for src, trg in test_loader:
            src = src.to(device).T
            trg = trg.to(device).T

            ys = seq2seq.beam_search_decode(src, max_len=40, beam_size=3)

            B = src.size(1)
            
            for b in range(min(B, n_show - shown)):
                src_txt = decode_ids(src[:, b], en_itos)
                trg_txt = decode_ids(trg[:, b], id_itos)
                pred_txt = decode_ids(ys[:, b], id_itos)
                
                references.append([trg_txt])
                hypotheses.append(pred_txt)

                if shown < n_show:
                    print("-" * 60)
                    print("SRC :", src_txt)
                    print("TRG :", trg_txt)
                    print("PRED:", pred_txt)
                    shown += 1
            
            if shown >= n_show:
                break
    
    bleu_final = sacrebleu.corpus_bleu(hypotheses, references).score
    print(f"Final BLEU score: {bleu_final:.2f}")

if __name__ == "__main__":
    main()

Total usable pairs after filtering: 14,856
Train: 11,884, Val: 1,485, Test: 1,487
EN vocab size: 6,101 | ID vocab size: 6,884


100%|██████████| 47/47 [00:01<00:00, 30.55it/s]


Epoch 01 | TF=0.70 | Train Loss 4.4330 PPL 84.19 | Val Loss 5.2005 PPL 181.36 | Val Bleu 42.7287 
Saving best to bahdanau_best.pt


100%|██████████| 47/47 [00:01<00:00, 30.78it/s]


Epoch 02 | TF=0.66 | Train Loss 2.3116 PPL 10.09 | Val Loss 4.8305 PPL 125.27 | Val Bleu 63.8943 
Saving best to bahdanau_best.pt


...

Epoch 09 | TF=0.38 | Train Loss 0.4649 PPL 1.59 | Val Loss 5.7414 PPL 311.49 | Val Bleu 29.0715 


100%|██████████| 47/47 [00:01<00:00, 30.65it/s]


Epoch 10 | TF=0.34 | Train Loss 0.4535 PPL 1.57 | Val Loss 5.8773 PPL 356.83 | Val Bleu 29.0715 


100%|██████████| 47/47 [00:01<00:00, 32.83it/s]


TEST | Loss 6.1374 | PPL 462.83 | SacreBLEU 34.57
------------------------------------------------------------
SRC : i have something that i want to say to you.
TRG : ada hal yang ingin aku katakan padamu.
PRED: saya ada yang ingin aku ingin kuberitahukan
------------------------------------------------------------
SRC : i have to make the best of that small room.
TRG : aku harus memanfaatkan ruang yang kecil itu <unk>
PRED: saya harus pergi ke kamar kecil.
------------------------------------------------------------
SRC : i have to <unk> some cash from the bank.
TRG : saya perlu mengambil uang di bank.
PRED: saya harus menyelesaikan dari dari sepupu
------------------------------------------------------------
SRC : i just want to know what actually happened.
TRG : aku hanya ingin mengetahui apa yang sebenarnya terjadi.
PRED: aku hanya ingin tahu apa yang yang terjadi.
------------------------------------------------------------
SRC : i just wanted to check to see if you're ok.
TRG : a